In [10]:
import os
import pandas as pd
import pyodbc

import numpy as np 
import matplotlib.pyplot as plt

from sklearn.linear_model import ElasticNet
from sklearn.datasets import make_regression
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import seaborn as sns

Accessing SQL database for the ML model

In [11]:
server = 'gen10-data-fundamentals-22-02-sql-server.database.windows.net'
database = 'guinea-pigs-database'
username = 'gPigs'
password = 'GuineaPigs!'
driver='{SQL Server}'

cnxn = pyodbc.connect('DRIVER='+driver+';SERVER='+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()
cursor.execute("SELECT * FROM NonNormalizedData")
rows = cursor.fetchall()
c = [column[0] for column in cursor.description]
ma_df = pd.DataFrame([tuple(t) for t in rows], columns = c) 
ma_df

,Approved Amount Content,Approved Amount EIDL,Approved Amount Real Estate,County,Damaged Property Zip Code,Fines and Forfeits,Interest Revenue,Market Value,Property Tax,SBA Disaster Number,SBA EIDL Declaration Number,SBA Physical Declaration Number,State,Timestamp,Total Salaries & Wages,Unemployement,Verified Loss Content,Verified Loss Real Estate,Year,_c0
0,0.0,1100.0,8500.0,BURLEIGH,58501.0,1873,6011,935.9449999999999,40719,ND-00016,11708.0,11707,NORTH DAKOTA,1652797757432,43317,3.5,0.0,42212.0,2009,2922
1,348200.0,0.0,0.0,DELAWARE,19023.0,0,31,1277.77,1513,PA-00042,12821.0,12820,PENNSYLVANIA,1652797764108,760,7.8,403163.0,96278.0,2011,3356
2,0.0,290500.0,10900.0,MONMOUTH,7724.0,673,2892,1646.49,65472,NJ-00033,13368.0,13367,NEW JERSEY,1652797768723,247125,7.5,0.0,13531.0,2013,2303
3,0.0,78700.0,0.0,CASS,58078.0,978,1078,935.9449999999999,4581,MN-00020,11710.0,11709,NORTH DAKOTA,1652797757434,18062,4.1,0.0,0.0,2009,2930
4,3400.0,0.0,132300.0,DELAWARE,19013.0,0,31,1277.77,1513,PA-00042,12821.0,12820,PENNSYLVANIA,1652797764110,760,7.8,7349.0,132208.0,2011,3356
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1903,14000.0,0.0,12900.0,ATLANTIC,8225.0,1440,1157,1646.49,95506,NJ-00033,13368.0,13367,NEW JERSEY,1652797766253,195336,11.6,181807.0,2873201.0,2013,2241
1904,0.0,0.0,0.0,ATLANTIC,8232.0,1440,1157,1646.49,95506,NJ-00033,13368.0,13367,NEW JERSEY,1652797766255,195336,11.6,122967.0,0.0,2013,2241
1905,2500.0,29500.0,30600.0,ATLANTIC,8330.0,1440,1157,1646.49,95506,NJ-00033,13368.0,13367,NEW JERSEY,1652797766257,195336,11.6,75248.0,77620.0,2013,2241
1906,132800.0,127000.0,482800.0,ATLANTIC,8402.0,1440,1157,1646.49,95506,NJ-00033,13368.0,13367,NEW JERSEY,1652797766260,195336,11.6,373872.0,2367927.0,2013,2241


Defining 'X' and 'y' variables. We are predicting Unemployment rate, so we are using that column as our 'y' value. Our 'X' value will be only the numerical data without the unemployment rate.


Executing the train test split method to split our dataset to be used for training and then testing the model. 

In [12]:
y = ma_df['Unemployement']
X = ma_df.drop(['Unemployement', 'County', 'SBA Disaster Number', 'State', 'Timestamp'], axis=1.)

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.25, random_state=0)

In [13]:
model = ElasticNet(alpha=1.0, l1_ratio=0.5)
ma_enet_model = model.fit(X_train, y_train)

In [14]:
ma_enet_model.coef_ 

array([ 3.72462786e-07, -3.96906071e-07, -3.80204528e-07, -1.84250094e-05,
       -1.41723373e-05, -4.87706431e-05, -8.28176626e-03,  5.80474588e-06,
        2.45762935e-03,  1.59038620e-07,  1.75147098e-08, -1.77983243e-07,
        1.39562491e-07,  0.00000000e+00, -1.00037589e-03])

In [15]:
ma_enet_model.intercept_

-8.819673090031685

In [16]:
train_prediction = ma_enet_model.predict(X_train)[:10]

train_prediction


array([7.37491969, 8.45995704, 9.17570008, 8.44928536, 9.67672978,
       7.63014486, 8.35180714, 7.94205351, 9.11768966, 9.59226427])

In [17]:
model_prediction = ma_enet_model.predict(X_test)[:10]

model_prediction

array([8.00453679, 9.66333564, 9.73857481, 7.63968473, 8.18259102,
       9.12300646, 8.45683724, 9.24583476, 8.04296312, 5.50713182])